## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kruisfontein,ZA,-34.0033,24.7314,59.88,87,100,12.08,moderate rain
1,1,Butaritari,KI,3.0707,172.7902,80.69,70,38,12.15,scattered clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,48.31,71,0,16.11,clear sky
3,3,Clyde River,CA,70.4692,-68.5914,-3.77,77,100,24.16,light snow
4,4,Zhigansk,RU,66.7697,123.3711,10.67,84,49,5.48,scattered clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(3)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kruisfontein,ZA,-34.0033,24.7314,59.88,87,100,12.08,moderate rain
6,6,Bluff,NZ,-46.6000,168.3333,60.39,74,18,9.40,few clouds
9,9,Bredasdorp,ZA,-34.5322,20.0403,62.62,64,100,12.82,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        186
City           186
Country        179
Lat            186
Lng            186
Max Temp       186
Humidity       186
Cloudiness     186
Wind Speed     186
Description    186
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()

#dropping any null values
clean_df.dropna(inplace=True)

# checking new data frame
clean_df.head(3)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kruisfontein,ZA,-34.0033,24.7314,59.88,87,100,12.08,moderate rain
6,6,Bluff,NZ,-46.6000,168.3333,60.39,74,18,9.40,few clouds
9,9,Bredasdorp,ZA,-34.5322,20.0403,62.62,64,100,12.82,overcast clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Kruisfontein,ZA,59.88,moderate rain,-34.0033,24.7314,
6,Bluff,NZ,60.39,few clouds,-46.6000,168.3333,
9,Bredasdorp,ZA,62.62,overcast clouds,-34.5322,20.0403,
12,Lebu,CL,54.64,clear sky,-37.6167,-73.6500,
24,Ponta Delgada,PT,59.72,scattered clouds,37.7333,-25.6667,
26,Haimen,CN,62.49,overcast clouds,23.1935,116.6142,
32,Hermanus,ZA,58.80,overcast clouds,-34.4187,19.2345,
34,Presidencia Roque Saenz Pena,AR,57.20,clear sky,-26.7852,-60.4388,
36,Mount Gambier,AU,64.31,overcast clouds,-37.8333,140.7667,
41,Salalah,OM,69.89,clear sky,17.0151,54.0924,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
# Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()

clean_hotel_df.dropna(inplace=True)

clean_hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))